In [1]:
from postprocessing import *
import numpy as np
import time, os, random, shutil, pickle, ray
from cgp import *
import matplotlib.pyplot as plt
from configuration import config
import warnings
warnings.filterwarnings('ignore')

In [26]:
def rollout(dataset, policy):
    x, y, z = dataset
    count = 0
    for xi,yi,zi in zip(x,y,z):
        # input = [d[0], d[1]]
        # label = d[2]
        input = [xi, yi]
        output = policy.eval(*input)
        if type(output)!=float:
            continue
        # print(zi, output, zi == output)
        # if zi == output:
        #     count += 1
        count += np.abs(zi-output)
    return count / len(x)

def rollout2(dataset, policy):
    x, y = dataset
    z = [xx>=1.1*yy for xx,yy in zip(x,y)]
    count = 0
    for xi, yi, zi in zip(x,y,z):
        input = [xi, yi]
        output = policy.eval(*input)
        if output == zi:
            count += 1
        # print(zi, output, zi == output)
        # if zi == output:
        #     count += 1
        
        
        # count += np.abs(zi-output)
    return count / 1000

In [27]:
def get_data():
    num = 100
    dim = 10
    x, y, z = [], [], []
    for i in range(num):
        xi = np.random.rand(dim,)
        x.append(xi)
        yi = xi * (np.random.uniform([-0.15]*dim, [0.15]*dim) + 1)
        y.append(yi)
        
        zi = np.sum(np.sign(np.maximum((yi-xi)/xi-0.1, 0))*xi)
        z.append(zi)
        
    return [x, y, z]

def get_data2():
    num = 1000
    train, label = [], []
    for i in range(num):
        x = np.random.uniform(-100,100)
        y = x * (1+np.random.uniform(-0.2, 0.2))
        train.append(x)
        label.append(y)
    return [train, label]

In [28]:
fs = [
        # Function(const_01, 0),
        # Function(op.add, 2), 
        Function(op.sub, 2), 
        # Function(op.mul, 2), 
        # Function(protected_div, 2),
        # Function(op.neg, 1),
        Function(op.abs, 1),
        # Function(op.ge, 2),
        # Function(op.le, 2),
        Function(sign, 1),
        Function(tenth, 1),
    ]
def Abs(x): return np.abs(x)
new_functions = [
    # Function(Add, 2, name='Add'),
    Function(Sub, 2, name='Sub'),
    # Function(Mul, 2, name='Mul'),
    # Function(Div, 2, name='Div'),
    # Function(Neg, 1, name='Neg'),
    Function(Abs, 1, name='Abs'),
    # Function(Max1, 1, name='Max2'),
    # Function(Sum, 1, name='Sum'),
    # Function(Min1, 1, name='Min'),
    Function(Sign, 1, name='Sgn'),
    Function(tenth, 1,)
    # Function(const_01, 0, name='0.1')
]

In [29]:
np.random.seed(123)
random.seed(123)
pop = create_population(config.MU+config.LAMBDA, 
                            input_dim=2, 
                            out_dim=1,
                            fs=fs,
                            # fs=new_functions,
                            n_cols=10,
                            level_back=5,
                            out_random_active=False)
# data = get_data()
data = get_data2()

In [30]:
for g in range(config.N_GEN):
    tick = time.time()
    fit_list = [rollout2(data, p) for p in pop]
    # fitness = ray.get(fit_list)
    # 优化方向是最大化
    for f, p in zip(fit_list, pop):
        p.fitness = f
    pop = evolve(pop, config.MUT_PB, config.MU, config.LAMBDA)
    print(g, 'time:', round(time.time()-tick, 3),'best fitness:', pop[0].fitness)

0 time: 0.203 best fitness: 0.723
1 time: 0.215 best fitness: 0.723
2 time: 0.196 best fitness: 0.723
3 time: 0.182 best fitness: 0.738
4 time: 0.202 best fitness: 0.784
5 time: 0.218 best fitness: 0.809
6 time: 0.153 best fitness: 0.809
7 time: 0.181 best fitness: 0.809
8 time: 0.174 best fitness: 0.809
9 time: 0.196 best fitness: 0.809
10 time: 0.172 best fitness: 0.809
11 time: 0.154 best fitness: 0.809
12 time: 0.161 best fitness: 0.809
13 time: 0.238 best fitness: 0.809
14 time: 0.199 best fitness: 0.809
15 time: 0.195 best fitness: 0.809
16 time: 0.26 best fitness: 0.809
17 time: 0.268 best fitness: 0.809
18 time: 0.252 best fitness: 0.809
19 time: 0.254 best fitness: 0.981
20 time: 0.15 best fitness: 1.0
21 time: 0.155 best fitness: 1.0
22 time: 0.168 best fitness: 1.0
23 time: 0.169 best fitness: 1.0
24 time: 0.153 best fitness: 1.0
25 time: 0.15 best fitness: 1.0
26 time: 0.14 best fitness: 1.0
27 time: 0.145 best fitness: 1.0
28 time: 0.139 best fitness: 1.0
29 time: 0.149 be

KeyboardInterrupt: 

In [31]:
g = extract_computational_subgraph(pop[0])
visualize(g, to_file='test.png',input_names=['x','y'])

Operator notation of 'tenth'' is not available. The node id is shown instead.
Operator notation of 'sign'' is not available. The node id is shown instead.


In [12]:
pop[0]